In [27]:
# https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import sys

In [28]:
raw_text = open('Output.txt').read().lower()

In [29]:
# map characters to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [30]:
n_chars = len(raw_text)
n_vocab = len(chars)
print (n_chars, n_vocab)

91641 52


In [31]:
# define training data for th enetwork
# split text into 100 characters (arbitrary)

seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print(n_patterns)

91541


In [32]:
# 1 transform list of input seq into form [samples, time steps, features]
# 2 rescale integers to range 0 to 1
# 3 convert output patterns into one encoding so each char has a probability value

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize 
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)


In [33]:
# define LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [34]:
# define checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [35]:
# fit model to data
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
91541/91541 [==============================] - 478s 5ms/step - loss: 2.9524

Epoch 00001: loss improved from inf to 2.95242, saving model to weights-improvement-01-2.9524.hdf5
Epoch 2/20
91541/91541 [==============================] - 794s 9ms/step - loss: 2.7150

Epoch 00002: loss improved from 2.95242 to 2.71502, saving model to weights-improvement-02-2.7150.hdf5
Epoch 3/20
91541/91541 [==============================] - 459s 5ms/step - loss: 2.5967

Epoch 00003: loss improved from 2.71502 to 2.59672, saving model to weights-improvement-03-2.5967.hdf5
Epoch 4/20
91541/91541 [==============================] - 472s 5ms/step - loss: 2.4871

Epoch 00004: loss improved from 2.59672 to 2.48707, saving model to weights-improvement-04-2.4871.hdf5
Epoch 5/20
91541/91541 [==============================] - 477s 5ms/step - loss: 2.3743

Epoch 00005: loss improved from 2.48707 to 2.37433, saving model to weights-improvement-05-2.3743.hdf5
Epoch 6/20
91541/91541 [=========================

In [10]:
# generate the text
# the exciting part!

In [36]:
# load the network weights
filename = "weights-improvement-20-1.1968.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [37]:
# reverse mapping for human eyeballs to read
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [41]:
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(500):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" would be a good teacher.

and so, our treatise will comprise all of our knowledge, as a book

decora "
ene on tee 
andaav on him thr walh the tooer nf moe and th in toil iote aitadt ano tithits.



    	
  
io pu thioty uo the mrher  to call ti lee liru.



    	
  
i aa the surrd guard on hhr hamd  you hl gete an andadt.

i am the suart of byor daoge  and you'll be lllt weuh the wword 



   
      


   
 



  

        

        


                                
         
                                                                                                                        
Done.
